In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from fastai.vision.all import *

In [ ]:
# Path to the data

TRAIN_FILE_PATH = '/kaggle/input/cassava-leaf-disease-classification/train_images'
TRAIN_CSV_PATH = '/kaggle/input/cassava-leaf-disease-classification/train.csv'
SUBMISSION_PATH = '/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv'

In [ ]:
train_df = pd.read_csv(TRAIN_CSV_PATH)
sub_df = pd.read_csv(SUBMISSION_PATH)
train_df.head()
sub_df.head()

In [ ]:
# Inspecting the target distribution
import seaborn as sns
train_df['label'].value_counts(normalize=True)
sns.countplot(train_df['label'])

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, x, y = train_test_split(train_df['image_id'], train_df['label'], stratify=train_df['label'], test_size=0.3, random_state=11) 
val_idxs = val_X.index
train_idxs = train_X.index

In [ ]:
item_tfms = Resize(512)
# Image augmentation
batch_tfms = [RandomResizedCrop(size=224), *aug_transforms(mult=1.0, do_flip=True, max_rotate=30.0),
                                    Normalize.from_stats(*imagenet_stats)]


dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                    splitter = IndexSplitter(val_idxs),
                    get_x = ColReader(0,pref=f'{TRAIN_FILE_PATH}/',suff=''),
                    get_y = ColReader(1, pref=f'{TRAIN_FILE_PATH}/'),
                    item_tfms = item_tfms,
                    batch_tfms = batch_tfms)

In [ ]:
dls = dblock.dataloaders(train_df,bs=64)

In [ ]:
dls.show_batch()


In [ ]:
# mixup
mixup=MixUp(0.4)
model = xresnet18()
learn = Learner(dls, model,loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)

In [ ]:
#learn.lr_find()

In [ ]:
learn.fit_one_cycle(20, 3e-3, cbs=mixup)

In [ ]:
#
TEST_FILE_PATH = '/kaggle/input/cassava-leaf-disease-classification/test_images'

In [ ]:
_sample_df = sub_df.copy()
test_files = TEST_FILE_PATH + '/' + _sample_df['image_id']
test_dl = dls.test_dl(test_files)


In [ ]:
# getting our prediction
preds, _ = learn.tta(dl=test_dl, n=3)

In [ ]:
sub_df['label'] = preds.argmax(dim=-1).numpy()

In [ ]:
sub_df.to_csv('submission.csv', index=False)